In [2]:
import cirq
import numpy as np
from math import pi
#from cirq.ops import gate_operation
from cirq.value import Duration

#libraries to plot histogram
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

#google
from cirq import circuits, ops, sim, study, protocols, optimizers
from cirq.google import convert_to_xmon_gates
from cirq import circuits, ops, sim, study, protocols
from cirq.google import xmon_device

shots=1000

# grab the device
foxtail = cirq.google.Foxtail

# get the qubits of foxtail
qubits = sorted(list(foxtail.qubits))

N=2

In [4]:
def circuit(j,string):  
       
    circuit = cirq.Circuit(device=foxtail)            
    
    circuit.append(cirq.H(qubits[0]))

    for i in range(N-1):    
        circuit.append(cirq.CNOT(qubits[i],qubits[i+1]))
        
    
    #insert U gate on all qubits
    phi = (pi*j)/(N+1)
    #A  unitary 1-qubit gate U 
    matrix=np.array([[1, 0], [0, np.exp(1.0j*phi)]])
    # matrix=np.array([[np.exp(-1.0j*phi), 1], [1, np.exp(1.0j*phi)]])
    moment_0 = cirq.Moment( cirq.SingleQubitMatrixGate(matrix).on(qubits[q]) for q in range(N) ) 
    # moment_0 = cirq.Moment( cirq.PhasedXPowGate().on(qubits[q]) for q in range(N) ) 
    circuit.append(moment_0)
    
    
    for i in reversed(range(N-1)):
        circuit.append(cirq.CNOT(qubits[i],qubits[i+1]))
    
    circuit.append(cirq.H(qubits[0]))
        
    #measurement
    circuit.append(cirq.measure(*qubits, key='x'))      

    #print(circuit)
        
    return circuit

In [5]:
def simulation(circuit,string):

    if(string == 'NoNoise'):
            
        f = open("counts_NoNoise.txt","a+")
        simulator = cirq.Simulator()
        results = simulator.run(circuit, repetitions=shots)
        h = results.histogram(key='x') 
        #print(results)
        print(1/shots*h[0])

        f.write('{}'.format(1/shots*h[0]))
        f.write(',')
        f.close()  
        
    if(string == 'Noise'):
        
        f = open("counts_Noise.txt","a+")
    #Density simulator
        #simulator = cirq.DensityMatrixSimulator()           
        #results = simulator.run(circuit, repetitions=shots)
        #results = simulator.simulate(circuit) #per accedere alla matrice densità
        #print(np.around(results.final_density_matrix, 3)) #per accedere alla matrice densità
        
    #Generic simulator
        results = cirq.sample(circuit, repetitions=shots) 
        h = results.histogram(key='x') 
        #print(results)
        print(1/shots*h[0])

        f.write('{}'.format(1/shots*h[0]))
        f.write(',')
        f.close()  
        

In [6]:
def main ():    

    for j in range(2*N+1+1):   # for j in range(2*N+1):
        print('Histogram j = {}'.format(j))
        #simulation(circuit(j, 'NoNoise'), 'NoNoise')
        simulation(circuit(j, 'Noise'), 'Noise')
        

In [7]:
if __name__ == '__main__':
    main()

Histogram j = 0


ValueError: Unsupported gate type: cirq.SingleQubitMatrixGate(np.array([[(1+0j), 0j], [0j, (1+0j)]]))